In [2]:
### Building GPT from scratch from Andrej karthpathy make more series
import transformers

ModuleNotFoundError: No module named 'transformers'

In [3]:
import os
import torch

In [4]:
# read it in to inspect it
os.chdir('/mnt/data/projects/.immune/Personal/Transformers_Tutorial/')
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print(text[:1000])  # print the first 1000 characters to check

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("All the unique characters:", ''.join(chars))
print("Vocab size:", vocab_size)

All the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [7]:
# creating a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [8]:
# lets encode the entire text into the integer representation
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.type)
print(data[:1000])  # first 1000 characters encoded as integers

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f3df28734c0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [9]:
# splitting into train and val sets
n = int(0.9*len(data))  # first 90% will be
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8  # context length for predictions
train_data[:block_size+1] ### till 8 it will be train and 9th character is the target

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
print(decode(train_data[:block_size+1].tolist()))  # checking if decoding works

First Cit


In [12]:
x = train_data[:block_size]  # input
y = train_data[1:block_size+1]  # target
for t in range(block_size):
    context = x[:t+1]  # the context grows with t
    target = y[t]     # the target is always the next character
    print(f'when input is {context} the target: {target}')
    print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

when input is tensor([18]) the target: 47
when input is 'F' the target: 'i'
when input is tensor([18, 47]) the target: 56
when input is 'Fi' the target: 'r'
when input is tensor([18, 47, 56]) the target: 57
when input is 'Fir' the target: 's'
when input is tensor([18, 47, 56, 57]) the target: 58
when input is 'Firs' the target: 't'
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is 'First' the target: ' '
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is 'First ' the target: 'C'
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is 'First C' the target: 'i'
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is 'First Ci' the target: 't'


In [13]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

train_data.shape, val_data.shape
data = train_data
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
y - torch.stack([data[i+1:i+block_size+1] for i in ix])
print("x:", x)
print("y:", y)

x: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [14]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # random starting indices
    x = torch.stack([data[i:i+block_size] for i in ix])  # input sequences
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # target sequences
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context} the target: {target}')
        print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target: 43
when input is 'L' the target: 'e'
when input is tensor([24, 43]) the target: 58
when input is 'Le' the target: 't'
when input is tensor([24, 43, 58]) the target: 5
when input is 'Let' the target: "'"
when input is tensor([24, 43, 58,  5]) the target: 57
when input is "Let'" the target: 's'
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is "Let's" the target: ' '
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is "Let's " the target: 'h'
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input i

In [15]:
# Constructing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # this is random initialization which will be learned during training.

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

m = BigramLanguageModel(vocab_size)
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 65)
)
logits: tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [16]:
torch.zeros((1, 1))

tensor([[0.]])

In [17]:
logits.shape

torch.Size([32, 65])

In [18]:
token_embedding=nn.Embedding(vocab_size, vocab_size)
token_embedding.weight

Parameter containing:
tensor([[ 0.6243, -0.1004,  0.2456,  ..., -1.2338,  0.0534, -0.1949],
        [ 0.8603, -0.2675, -0.4354,  ..., -0.1845, -1.0497, -0.8261],
        [ 1.7343,  0.8752,  1.5813,  ...,  1.2927,  0.7182,  0.6097],
        ...,
        [-1.6689, -3.3766, -0.4560,  ...,  0.2780, -1.5953, -0.4737],
        [-1.2969,  1.8702, -1.0598,  ..., -1.2579,  0.1119,  0.3389],
        [-0.0886, -1.0879,  0.9975,  ..., -1.2604,  0.8044, -1.9819]],
       requires_grad=True)

In [19]:
logits = token_embedding(xb) # (B,T,C)

In [20]:
print(logits[0])
print(loss)
print(xb.shape) # B X T
print(yb.shape) # B X T

tensor([[-1.3660e-01, -1.8731e+00,  7.1989e-01,  1.3609e-01,  9.8589e-01,
          1.5483e+00, -2.6414e-01, -1.1776e+00,  5.1714e-01, -4.6053e-01,
          4.2017e-01, -9.6025e-01,  1.3287e+00,  2.0909e-01,  8.6215e-01,
          4.3085e-02,  5.7950e-01, -2.0212e+00, -8.1185e-01,  6.5908e-01,
         -7.5360e-02,  1.3957e+00,  2.0643e+00, -2.0412e+00, -3.7620e-01,
         -3.8499e-01,  3.5262e-01, -4.5423e-01,  1.1916e+00, -8.5033e-01,
         -1.1318e+00,  9.0647e-01,  5.6884e-01, -2.9375e+00,  4.6553e-01,
         -9.6770e-01, -1.4652e+00, -2.6071e-01, -1.0342e+00, -1.3620e+00,
         -1.2595e+00, -1.8476e+00,  1.1339e+00, -2.0390e-01,  2.8299e-01,
          3.1425e-01,  1.5377e+00,  9.7995e-02,  1.5568e-01, -3.9144e-01,
         -1.0180e+00,  2.9127e-01, -1.4972e+00,  8.3221e-01,  1.5373e+00,
         -1.1809e+00,  2.8087e+00,  1.7374e+00, -5.5859e-01, -8.0195e-01,
         -5.1946e-01, -9.9884e-01,  2.1045e-01,  8.7487e-01,  2.4380e-01],
        [ 4.1719e-01,  6.3408e-01, -1

Training the model

In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
### This is mainly learning from the probability of the look up table
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # optimize the model
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.6904830932617188


In [23]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ss d tcI:
AUNorthiref s is, XLANzncouso;AUCl
AHllico ithen, Horeo t had,

Mh
FINolly



Uy ou ttirve
' ll ato, s then w; s w
EQzL:CEXR:
Whas nsu bfof; cond,

IFtrune e fo wa at we al
Cid thind ad cas;
Youtod,
Fo s mp aSwhy'lout.
RFXWathininoth itethe t r; terus rs t;
vell COUCENET:
bints tes?$a:
OLk'douid ours;
p' w'raver:

e,
AUA c,
UcousexFIEOr orous sthVIOLinr om!
STEDICHe kenf tis bau d IU horf t q.
bur,
Issthe; rinodvely,V&ven tang, horche.?
Thaceayousomege, thage pe'se.
ORYQ!ve
JFVbep On s


## The mathematical trick in self-attention

In [24]:
## This is for 2 dimesnional matrix multiplication
# Since in transformer we will be using 2D matrices for attention calculations.
# In attention we use only those character before the current character for prediction.
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # Lower triangular matrix
a = a / torch.sum(a, dim=1, keepdim=True) # scaling the rows to sum to 1
print(a)
b = torch.randint(0, 10, (3, 2)).float()
print(b)
c = a @ b
print(c) 


tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
## This is for 3 dimesnional matrix multiplication
torch.manual_seed(42)
a = torch.tril(torch.ones(3,4,3)) # Lower triangular matrix
a = a / torch.sum(a, dim=1, keepdim=True) # scaling the rows to sum to 1
# print(a)
print(a.shape)
b = torch.randint(0, 10, (3, 2)).float()
# print(b)
print(b.shape)
c = a @ b
# print(c)
print(c.shape)
# so regarding the dimensions, a = (3,4,3) and b = (3,2) results in c = (3,4,2)
# since b = (3,3,2) and while matrix multiplication the last two dimensions are considered for multiplication.
# so a = (4,3) and b = (3,2) results in c = (4,2)


torch.Size([3, 4, 3])
torch.Size([3, 2])
torch.Size([3, 4, 2])


In [26]:
# try out on our dataset
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape
print(x[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [27]:
### In the self attention here we are just averaging the previous tokens to get a sense of context.

xbow = torch.zeros((B, T, C))
print(xbow.shape)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev,0) # averaging the previous tokens including the current token
print(xbow.shape)
print(xbow[0])

torch.Size([4, 8, 2])
torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [28]:
### Another more efficient way
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)  #
print(wei)
xbow2 = wei @ x  # (T,T) @ (T,C) -->
print(xbow2.shape)
print(xbow2[1])
print(xbow[1])
torch.allclose(xbow[0], xbow2[0]) ### it is same

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
torch.Size([4, 8, 2])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0

True

In [29]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [30]:
wei = torch.zeros((T,T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [31]:
### how much does the previous elements contribute to the current element. 
# so we can use the softmax which is kind of the normalization
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
wei = F.softmax(wei, dim = -1)
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [32]:
# Constructing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        token_emb = self.token_embedding_table(idx)  # (B,T,C)
        logits = self.lm_head(token_emb)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32
batch_size = 32
m = BigramLanguageModel()
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)
logits: tensor([[ 0.6532, -0.5768,  0.1623,  ...,  0.9086, -0.3203, -0.5369],
        [ 0.4420,  0.1231,  0.8807,  ...,  0.5617,  0.1265,  0.2913],
        [ 1.0582, -0.5045,  0.3036,  ..., -0.3142,  0.5471,  0.7962],
        ...,
        [-0.1580, -0.4475,  0.3257,  ...,  0.1921,  0.7424,  0.4570],
        [ 0.4546, -0.3635,  0.2983,  ...,  0.2652,  0.5731,  0.0840],
        [-0.6873, -0.4842,  0.6730,  ..., -0.5416, -0.7946, -0.7934]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.2468, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

lN!BJ'kysLCMFJPKOL?DP-QWwrEoL?jLDJQOL.f'RIHD'Hdhs Yv,wxatnscMZwtEOS'palkq3ssZeAvzF-QT;eMk;x.gQSFCLgx


In [33]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [34]:
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5342628955841064


In [35]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Th thalldy hangilyoteng h hasbe pan hatrance
RDe hicomyonthar's
PES:
AKEd ith henoungincenonthiousir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprrouthercckeha t,
K:

My hind tt hinig t ouchos tes; st younind wotte grotonear 'so it t jod weancothanan hay. t--s n prids, r loncave w hollular e O:
HIs; ht anje caike ineent.

Lavinde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, aridesar, wyonthenous aves, theresseys
Plorseelapinghiybend yof GLANCHI me. strsithisgothers w de o! ABe


#### Adding the Position Embedding to it

In [36]:
xb.shape

torch.Size([32, 8])

In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

## First lets initialize 
vocab_size = 65 # since in the Shakespeare vocabulary has 65 characters 
emb_size = 128 # total number of dimensions
batch_size = 32 # batch size
token_size = 8
## It is just randomly creating an embedding for all the vocabulary and Tokens with 128 dimensions
token_embedding_table = nn.Embedding(vocab_size, emb_size) 
token_position_table = nn.Embedding(token_size, emb_size)
lm_head = nn.Linear(emb_size, vocab_size) 

In [38]:
print("character embedding")
print(token_embedding_table)
print(token_embedding_table.weight[0,:].shape)
# print(token_embedding_table.weight[0,:] )
# This shows for first character what is the weight will keep on updating during the backpropogation
print("position embedding")
print(token_position_table)
print(token_position_table.weight[0,:].shape)
# print(token_position_table.weight[0,:])
# This shows for first token position what is the weight will keep on updating during the backpropogation
print("linear Model")
print(lm_head)
print(lm_head.weight[0,:].shape)
# lm_head.weight[0,:]


character embedding
Embedding(65, 128)
torch.Size([128])
position embedding
Embedding(8, 128)
torch.Size([128])
linear Model
Linear(in_features=128, out_features=65, bias=True)
torch.Size([128])


In [39]:
parameters = [
        token_embedding_table.weight,
        token_position_table.weight,
        lm_head.weight,
        lm_head.bias
        ]


In [ ]:
### After initialization, we will input the values
# print("Input=",xb[0:1:10], xb.shape,"\nTarget=",yb[0:1:10], yb.shape)
# Now we have to embed these tokens
# token_character embedding which will provide us Batch, Token, Embedding 32, 8, 128 (B,T,C)

for steps in range(10000):
    B,T = xb.shape
    token_emb= token_embedding_table(xb)
    # print("Input_embdding=",token_emb.shape)
    position_emb= token_position_table(torch.arange(T)) ## Since we have to take care of the position
    # print("Input_pos_embdding=",position_emb.shape)
    x = token_emb + position_emb
    # print("x=",x.shape)
    logits = lm_head(x) # xW.transpose() + bias
    B,T,C = logits.shape
    logits3 = logits.view(B*T,C)
    # print("logits:",logits3.shape)
    targets = yb.view(B*T)
    # print('targets:',targets.shape)
    loss = F.cross_entropy(logits3, targets)
    # optimizer.zero_grad(set_to_none = True)
    # loss.backward()
    # optimizer.step()
    # Gradient reset
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update
    lr = 0.001
    for p in parameters:
        p.data += -lr * p.grad
    # logits2 = logits[:,-1,:]
    # print("logits=",logits2.shape)
    # prob = F.softmax(logits2, dim = -1)
    # print("prob=",prob.shape)
    # print(prob.sum(1))

print(loss.item()) # since we are training on single batch so it is overcorrecting it

1.400627851486206


In [41]:
def generate(idx, max_new_tokens):
    # idx: (B, T) tensor of token indices (context). Usually start with [[0]].
    for _ in range(max_new_tokens):
        B, T = idx.shape
        # Forward pass for only the current context
        token_emb = token_embedding_table(idx)                         # (B, T, C)
        position_emb = token_position_table(torch.arange(T)) # (T, C)
        x = token_emb + position_emb                                   # (B, T, C)
        logits = lm_head(x)                                            # (B, T, vocab)
        # take only the last time step
        logits = logits[:, -1, :]                                      # (B, vocab)
        # convert logits → probabilities
        probs = F.softmax(logits, dim=-1)                              # (B, vocab)
        # sample next token
        idx_next = torch.multinomial(probs, num_samples=1)             # (B, 1)
        # append to sequence
        idx = torch.cat((idx, idx_next), dim=1)                        # (B, T+1)
    return idx


In [42]:
start = torch.zeros((1, 1), dtype=torch.long)   # context = "unknown token" / start token
output = generate(start, max_new_tokens=8)
print(decode(output[0].tolist()))


. ,aIm,



In [43]:
context = torch.zeros((1, 1), dtype=torch.long)  # or start token / first character
generated = context.clone()

for _ in range(200):   # generate 200 characters
    # crop context to last 8 chars (model's block size)
    context_cropped = generated[:, -8:]

    token_emb = token_embedding_table(context_cropped)
    pos_emb = token_position_table(torch.arange(context_cropped.shape[1]))
    x = token_emb + pos_emb
    logits = lm_head(x)

    # take logits of last time step
    logits_last = logits[:, -1, :]
    probs = F.softmax(logits_last, dim=-1)

    # sample from distribution
    next_token = torch.multinomial(probs, num_samples=1)

    # append to generated sequence
    generated = torch.cat([generated, next_token], dim=1)

print(generated)


tensor([[ 0, 23, 13, 32, 20, 13,  1, 21,  1, 61, 47, 49, 43, 57,  1, 61, 47, 49,
         43,  1, 44, 43,  1, 53, 51, 22, 53, 51, 59, 56, 19, 53, 19, 53, 44, 44,
         39, 52, 58, 39, 52, 58, 46, 47, 58, 58, 39, 52, 54,  4, 26, 26, 53, 61,
         53, 51, 39, 52, 60, 43, 52, 58, 46, 43, 52, 58, 39,  0, 19, 19, 53, 20,
         53, 51, 39, 49, 43,  1, 26,  1, 63, 36, 51, 39, 52, 47, 53, 56, 47, 49,
         43, 19, 53, 44,  1, 51, 51, 39, 52, 42,  1, 46, 47, 41, 46, 47, 49, 43,
          1, 41, 47, 49, 43, 23,  1, 26, 39, 52, 45, 47, 57, 47, 53, 58, 46, 47,
         49, 43,  1, 26, 39, 52, 53, 44, 44, 47, 49, 43, 56, 41, 47, 57,  1, 51,
         41, 47, 49, 43,  1, 61, 47, 49, 43, 62, 22, 44, 41, 59, 54, 47, 39, 47,
         54, 64, 43,  1, 63,  1, 26, 39, 52, 54, 53, 51, 41, 46, 47, 58, 46, 47,
         49, 43,  1, 46, 11, 57,  1, 51, 39, 53, 61, 47,  1, 62, 52, 42,  1, 63,
         44, 44, 21]])


In [44]:
decoded = ''.join([itos[int(i)] for i in generated[0].tolist()])
print(decoded)



KATHA I wikes wike fe omJomurGoGoffantanthittanp&NNowomanventhenta
GGoHomake N yXmaniorikeGof mmand hichike cikeK Nangisiothike Nanoffikercis mcike wikexJfcupiaipze y Nanpomchithike h;s maowi xnd yffI


In [45]:
for step in range(10000):
    B, T = xb.shape
    token_emb = token_embedding_table(xb)                        # (B, T, C)
    position_emb = token_position_table(torch.arange(T))  # (T, C)
    x = token_emb + position_emb                                 # (B, T, C)

    logits = lm_head(x)                                          # (B, T, vocab)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = yb.view(B*T)

    loss = F.cross_entropy(logits, targets)

    # Gradient reset
    for p in parameters:
        p.grad = None

    # Backward
    loss.backward()

    # Update
    lr = 1e-3
    for p in parameters:
        p.data += -lr * p.grad

print(loss.item())


1.2227048873901367


In [46]:
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5143752098083496


In [47]:
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # cropping the idx so that in the position embedding it is not bigger than the expecting so we will crop it
            idx_crop = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32 # this is basically the dimensions size
batch_size = 32
m = BigramLanguageModel()
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (token_position_embedding_table): Embedding(8, 32)
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)
logits: tensor([[ 0.1846, -0.1045, -1.0437,  ..., -0.4779,  0.1244,  0.5709],
        [ 0.5446,  0.3983,  0.2476,  ...,  0.3400,  0.5947, -0.8438],
        [ 0.4003,  1.3123,  1.5617,  ...,  0.2886, -0.7267,  0.4469],
        ...,
        [ 1.0967,  0.7229,  0.1110,  ...,  0.0170,  1.9136,  0.7253],
        [ 0.5252, -0.7673, -0.1252,  ..., -0.2411,  0.9101,  1.4005],
        [-0.6442, -0.5236,  1.1791,  ..., -0.7567, -0.3258,  0.3364]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.4711, grad_fn=<NllLossBackward0>)
torch.Size([256, 65])


In [48]:
## Self Attention Mechanism, we require key, query, and values
# attention = softmax((q @ wT)/d**0.5) @ v
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch tokens channels (dimension)
x = torch.randn(B,T,C)

head_size = 16 # it means we have 2 heads 32 / 16 = 2

key = nn.Linear(C, head_size, bias = False) # C is the in-features and head_size = out_features
query = nn.Linear(C, head_size, bias = False) # key
value = nn.Linear(C, head_size, bias = False)

print("key:", key)
print("key weight:", key.weight.shape)
print("query:", query)
print("query weight:", query.weight.shape)
print("value:", value)
print("value weight:", value.weight.shape)

k = key(x) # B,16,T # x @ W.transpose Linear layer, where W has dimension C, head_size
q = query(x) # B,16,T

wei = q @ k.transpose(-2,-1)
print(wei[0])
tril = torch.tril(torch.ones(T,T))

wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei[0])
print("wei:",wei.shape)
wei = F.softmax(wei, dim = -1)
print(wei[0])
print("wei:",wei.shape)

v = value(x)
out = wei @ v
print("k shape:", k.shape)
print("q shape:", q.shape)
print("v shape:", v.shape)
print(out.shape)

key: Linear(in_features=32, out_features=16, bias=False)
key weight: torch.Size([16, 32])
query: Linear(in_features=32, out_features=16, bias=False)
query weight: torch.Size([16, 32])
value: Linear(in_features=32, out_features=16, bias=False)
value weight: torch.Size([16, 32])
tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)
tensor([[-1.7629,    -inf,    -

In [ ]:
### Writing a head module
# attention = softmax((q @ wT)/d**0.5)) @ v
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        self.key = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        self.value = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        # here tril is not a paramter it is a buffer
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        # self.head_size = head_size
    
    def forward(self, x):
        B,T,C = x.shape # B,T,C
        q = self.query(x) # B,H,T
        k = self.key(x) # B,H,T
        # compute attention score
        # ((q @ wT)/d**0.5))
        # wei = q @ k.transpose(-2,-1) * self.head_size**-0.5 # B,T,H @ B,H,T --> B,T,T
        wei = q @ k.transpose(-2,-1) * C**-0.5 # B,T,H @ B,H,T --> B,T,T
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1) # softmax((q @ wT)/d**0.5))
        v = self.value(x) 
        out = wei @ v # softmax((q @ wT)/d**0.5)) @ v
        return out


In [50]:
## Now we will add the self attention head to this bigram model
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        self.sa_head = Head(head_size)
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        x = self.sa_head(x) 
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # cropping the idx so that in the position embedding it is not bigger than the expecting so we will crop it
            idx_crop = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_crop) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32 # this is basically the dimensions size
batch_size = 32
head_size = 32
block_size = 8
m = BigramLanguageModel()
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
print(targets.shape) # (B*T, C)
# # torch.zeroes is for new line character so we can initialize the generation from it.
# # This is completely random generation as the model is untrained.


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (token_position_embedding_table): Embedding(8, 32)
  (sa_head): Head(
    (query): Linear(in_features=32, out_features=32, bias=False)
    (key): Linear(in_features=32, out_features=32, bias=False)
    (value): Linear(in_features=32, out_features=32, bias=False)
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)
logits: tensor([[ 0.2431,  0.0681, -0.3286,  ...,  0.2470, -0.2253,  0.0566],
        [-0.1222,  0.0324, -0.1054,  ...,  0.0134, -0.7218,  0.0294],
        [-0.0980, -0.0212, -0.0708,  ..., -0.1384, -0.6062, -0.0994],
        ...,
        [-0.3403, -0.0469,  0.3340,  ...,  0.0522, -0.5408, -0.0759],
        [-0.1054,  0.1624,  0.0327,  ..., -0.2002,  0.1835, -0.0298],
        [-0.0480,  0.1829, -0.1359,  ..., -0.4397,  0.2710, -0.0786]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.1673, grad_fn=<NllLossBackward0>)
torch.Size([256, 65])
torch.Size([256])


In [51]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3578009605407715


In [52]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


hadd wist. Be sothereanomman in etalll illd? fisot
Th lal theise,
pit ban, pray ind, cikigelu inecast afrro nodess

By nee ive nno bou mad'des twhep'y st bazel
TELONO:
Y d''Teath n'tsous migaer wofr woudl as lisen, hisee,
Keratoh e'ld, his ssn nide te! as-'lly sin chof thes 'dis asw ht honger vit oul you,
Lonccouroust,
G ga oblleandalelarminthaf Thont pare nd heand ay out sitout youn havegnellfor,
Ay to bol my, a my dsht muden ise sesinot imeadeags.

CERLe Ine ows seewer, yousten do fothedst wak


### Multihead Attention

In [53]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim = -1)


In [54]:
## Now we will add the self attention head to this bigram model
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        self.sa_heads = MultiHeadAttention(4, n_emb // 4) ## i.e. 4 head for 8 dimension i.e. 32 / 4 = 8
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        x = self.sa_heads(x) 
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # cropping the idx so that in the position embedding it is not bigger than the expecting so we will crop it
            idx_crop = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_crop) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32 # this is basically the dimensions size
batch_size = 32
head_size = 32
block_size = 8
m = BigramLanguageModel()
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
print(targets.shape) # (B*T, C)
# # torch.zeroes is for new line character so we can initialize the generation from it.
# # This is completely random generation as the model is untrained.


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (token_position_embedding_table): Embedding(8, 32)
  (sa_heads): MultiHeadAttention(
    (heads): ModuleList(
      (0-3): 4 x Head(
        (query): Linear(in_features=32, out_features=8, bias=False)
        (key): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
    )
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)
logits: tensor([[ 0.7121, -0.0878, -0.2714,  ...,  0.0534,  0.3479, -0.1187],
        [ 0.0012, -0.2142, -0.0085,  ..., -0.0389, -0.0339, -0.0025],
        [-0.0431, -0.3961,  0.0912,  ..., -0.3318,  0.1192, -0.1161],
        ...,
        [-0.2010,  0.3399, -0.2436,  ...,  0.1715,  0.2606, -0.1198],
        [-0.0052,  0.2361, -0.2004,  ...,  0.3149,  0.3627,  0.0131],
        [ 0.0593,  0.1619, -0.1682,  ...,  0.3164,  0.3835,  0.0134]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.1898, grad_fn=<NllLossBa

In [55]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.189296245574951


In [56]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


hadd wist. Be siths
you mman in epare, aris? merow
Ther let his? wpit bar, arearind, cize wit and;
Gelaser' noth
Me
By nothive nich out hat belt anp'y st bazel
Teetiser, de'll thon theearmighto ford woudis selisen, his my de'llowne's,
I bects, wis bee! ase may sin coof thee 'dipd
Aw hathonger vit ouldy:
Off bust doust,
Gugh obleeakeall are frrabe atne pire not
Yand ay out sith uplest have bell brorfordy bold.
MIG EDWIHARTeldeatis the in afire, payst shat bleay gis selwerlish Orent meevereds.

Ther dunrd mond youch fas prise wan,
Tore bak onessps my have dis loTu though math hain pil to uth,
Tand sleser,
Mwobor with nothy them, of mar cof. Handme toousem hiold not os bund.

Porempstay no's andt:
Sw ann of all of lights mend movis st?

ARWIR:
Alhe pas ime wis the tohe ord ofet clin; heme for soo,
Wish nor the you deacegir of my ame in no--w--wingivert, for ovingien be.

Dof wat prot prrow fram you the hen, halike ture wengh ther, to poy a forn theace:
Thand cfimient.

I vecarl,
Apill id

### Feed Forward Layer

In [57]:
class FeedForwardLayer(nn.Module):
    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, n_emb),
            nn.ReLU()
        )
    
    def forward(self,x):
        return(self.net(x))

In [58]:
## Now we will add the self attention head to this bigram model
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        self.sa_heads = MultiHeadAttention(4, n_emb // 4) ## i.e. 4 head for 8 dimension i.e. 32 / 4 = 8
        self.fwd = FeedForwardLayer(n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        x = self.sa_heads(x)
        x = self.fwd(x)
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # cropping the idx so that in the position embedding it is not bigger than the expecting so we will crop it
            idx_crop = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_crop) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32 # this is basically the dimensions size
batch_size = 32
head_size = 32
block_size = 8
m = BigramLanguageModel()
print(m)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
print(targets.shape) # (B*T, C)
# # torch.zeroes is for new line character so we can initialize the generation from it.
# # This is completely random generation as the model is untrained.


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (token_position_embedding_table): Embedding(8, 32)
  (sa_heads): MultiHeadAttention(
    (heads): ModuleList(
      (0-3): 4 x Head(
        (query): Linear(in_features=32, out_features=8, bias=False)
        (key): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
    )
  )
  (fwd): FeedForwardLayer(
    (net): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
    )
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)
logits: tensor([[-0.0157, -0.0976,  0.0721,  ..., -0.0107,  0.0436,  0.0568],
        [-0.0756, -0.0747,  0.1014,  ..., -0.0352, -0.0082,  0.0192],
        [-0.0959, -0.1556, -0.0010,  ..., -0.1381,  0.0197,  0.0535],
        ...,
        [-0.0951,  0.0787,  0.1760,  ..., -0.0212,  0.0320,  0.0368],
        [-0.0633,  0.0604,  0.2020,  ...,  0.0719, -0.0374, -0.0317],
  

In [59]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.149420738220215


In [60]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


Surlet ofsce'im nal-mapp;
Dupon now
The foall ich
ARICERSTERILEORICUC:
Mallingear:
His; bulist ther ins fran bre nanis therequoutt,
Fortonce sre, his of tin.
Mome,
Lorre prurnust nough by fnot yout not wont
LOUWeare.

CONIUS:
Cauld witan persbous
My met.

Dombare batoor!

PUESTONT:
Govirend
How of id ithoust poltbe sinoth fir,
Thyatizeserm.

Put it gre, glat?
An bithis hown at this! shionelr a for buthates.

KINGLERIRCLOT:
Noliel;
At ifeettoot held.

Clakerecest.

Sutherd, bith llimthe irs beaind lan agood for thoulke you.
Shid pus Lorints;
That beast sard.

RIORGLUS:
Mith Walind cures jlory theat and
COPERESTIY:
Thiusttorplestroone he the?
Coo poong ting he sipnoardlalist it yout joothavhllwerd maces:
I
Het
Loropeyser, milessole's the pist bont ot, im and? coled: therakerse, takee to les-tea ited adifulth them to of ald so the thim nebe sere,
Hepont!

Vin it.
Io have paidgafitery han, a man, whand you.

KINGHONY:
That, ther.

FIO:
Toweeou fais is suth in to the curtt ensssat truled t

### Block

In [79]:
### Writing a head module
# attention = softmax((q @ wT)/d**0.5)) @ v
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        self.key = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        self.value = nn.Linear(in_features = n_emb, out_features = head_size, bias = False)
        # here tril is not a paramter it is a buffer
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        # self.head_size = head_size
        self.dropout = nn.Dropout(dropout) # works as a regularlization where number of node from the layer has been dropout
    
    def forward(self, x):
        B,T,C = x.shape # B,T,C
        q = self.query(x) # B,H,T
        k = self.key(x) # B,H,T
        # compute attention score
        # ((q @ wT)/d**0.5))
        # wei = q @ k.transpose(-2,-1) * self.head_size**-0.5 # B,T,H @ B,H,T --> B,T,T
        wei = q @ k.transpose(-2,-1) * C**-0.5 # B,T,H @ B,H,T --> B,T,T
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1) # softmax((q @ wT)/d**0.5))
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v # softmax((q @ wT)/d**0.5)) @ v
        return out


In [80]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
        self.proj = nn.Linear(n_emb, n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
         out = torch.cat([h(x) for h in self.heads], dim = -1)
         out = self.proj(out)
         return out


In [81]:
class FeedForwardLayer(nn.Module):
    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4 * n_emb), ### This will increase the number of nodes when moving to the non-linear layer i.e. ReLU
            nn.ReLU(),
            nn.Linear(4 * n_emb, n_emb), ### Here we come back to the same dimension
            nn.Dropout(dropout)
        )
    
    def forward(self,x):
        return(self.net(x))

In [82]:
class Block(nn.Module):
    def __init__(self, n_emb, n_head):
        super().__init__()
        head_size = n_emb // n_head
        self.sa_heads = MultiHeadAttention(n_head, head_size) ## i.e. 4 head for 8 dimension i.e. 32 / 4 = 8
        self.fwd = FeedForwardLayer(n_emb)
        self.ln1 = nn.LayerNorm(n_emb) #### we will be adding the linear normalization also so it can make in gaussian distribution
        self.ln2 = nn.LayerNorm(n_emb)
    
    def forward(self, x):
        x = x + self.sa_heads(self.ln1(x)) ### adding the skip connection
        x = x + self.fwd(self.ln2(x))
        return x


In [107]:
## Now we will add the blocks to this bigram model
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
## This is decoder only transformer
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        # self.block = nn.Sequential(
        #     Block(n_emb, n_head),
        #     Block(n_emb, n_head),
        #     Block(n_emb, n_head),
        #     nn.LayerNorm(n_emb),
        # )
        # Cosmetic changes
        self.block = nn.Sequential(*[Block(n_emb, n_head) for _ in range(n_layer)])
        self.ln_norm = nn.LayerNorm(n_emb) # layer_normalization
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        # idx = idx.to(self.embedding.weight.device)
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        # print("idx:", idx.device, " embedding:", self.embedding.weight.device)
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T, device=idx.device)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        x = self.block(x)
        x = self.ln_norm(x)
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # cropping the idx so that in the position embedding it is not bigger than the expecting so we will crop it
            idx_crop = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_crop) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 384 # this is basically the dimensions size
batch_size = 64 # number of batches in one go
head_size = 32
max_iter = 5000
block_size = 256 ### context length
learning_rate = 3e-04
n_head = 6
n_layer = 6
dropout = 0.2
m = BigramLanguageModel()
print(m)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m = m.to(device)
print(device)
print("Model running on:", next(m.parameters()).device)
xb = xb.to(device)
print(xb.device)
print(yb.device)
yb = yb.to(device)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
print(targets.shape) # (B*T, C)

# # torch.zeroes is for new line character so we can initialize the generation from it.
# # This is completely random generation as the model is untrained.


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (token_position_embedding_table): Embedding(256, 384)
  (block): Sequential(
    (0): Block(
      (sa_heads): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (query): Linear(in_features=384, out_features=64, bias=False)
            (key): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (fwd): FeedForwardLayer(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, 

In [112]:
# create a PyTorch optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m = m.to(device)
print("Model running on:", next(m.parameters()).device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
## Training a model
for steps in range(max_iter):
    xb,yb = get_batch('train')
    xb = xb.to(device)   # move training batch to GPU
    yb = yb.to(device)
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

Model running on: cuda:0
1.1135485172271729


In [114]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long, device = device), max_new_tokens=5000)[0].tolist()))


hands your form, she was married
'Twixt sorrow; and here was never diverse;
Linely in borderous may party of this,
That, that dasband swallow'd me furthers to do,
Hold rest not thy country's bless, than grossess'd,
What mutinous is the palment discipulate?
Methinks I am too began?

FRIAR
MIANCE:
OR Isabel counsent to a truth of mind mockery.

KING EDWARD IV:
Mine army right, I am a poor true-violaty.

QUEEN ELIZABETH:
I cannot keep and deservice;
But seem I mach the subdue that majesty.

KING RICHARD II:
Exeor, thou cansures' things hath done that;
For break of death, blest whilst I say thee,
Thou art of a bolts, not bad and the subject.
Though artiest their noses in heaven closed
With thembles thy held too noble and will not
Then a husband's claim tears, and manifesty
To say the base and the son of dulgrees
Ere thou comfort the former croosile that answer
Shall begin a blemish shame to hide to besome
That lock the and horses serly woman's course.

ARCHBISHOP OF YORK:
What! O chaffedi

In [115]:
os.getcwd()

'/mnt/data/projects/.immune/Personal/Transformers_Tutorial'

In [ ]:
## Only model weights are saved
torch.save(m.state_dict(), "bigram_model.pt")
# later to load the model
# m = BigramLanguageModel()            # recreate the model architecture
# m.load_state_dict(torch.load("bigram_model.pt"))
# m.to(device)                         # move to GPU if needed
# m.eval()             

In [117]:
# Save the entire model
torch.save(m, "bigram_model_full.pt")
# later to load the model
# m = BigramLanguageModel()            # recreate the model architecture
# m.load_state_dict(torch.load("bigram_model.pt"))
# m.to(device)                         # move to GPU if needed
# m.eval()             

In [118]:
os.getcwd()

'/mnt/data/projects/.immune/Personal/Transformers_Tutorial'